In [ ]:
import os
import sys
import random
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.pyplot import imshow
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.layers import MaxPooling2D,Dense,Dropout,Conv2D,Flatten
from tensorflow.keras.models import Model
from PIL import Image

from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img

In [ ]:
@tf.function
def num_char_acc(y_true,y_pred):
    y_true = tf.convert_to_tensor(y_true)
    _,num = tf.split(y_true, [40,4], 1)
    _,NUM = tf.split(y_pred, [40,4], 1)
    num_char_true = tf.math.argmax(num,axis=-1,output_type=tf.int32)
    num_char_pred = tf.math.argmax(NUM,axis=-1,output_type=tf.int32)
    n =tf.cast(num_char_true==num_char_pred,tf.float32)
    return tf.reduce_mean(n)

@tf.function
def custom_acc(y_true,y_pred):

    y_true = tf.convert_to_tensor(y_true)
    a,b,c,d ,num = tf.split(y_true, [10,10,10,10,4], 1)
    true = [a,b,c,d]
    A,B,C,D,_ = tf.split(y_pred, [10,10,10,10,4], 1)
    pred = [A,B,C,D]
    num_char_true = tf.math.argmax(num,axis=-1,output_type=tf.int32)+ 1
    i = 0 
    n =tf.constant(0,tf.int32)
    while (i<4) :
        char_pred = tf.math.argmax(pred[i],axis=-1,output_type=tf.int32)
        char_true = tf.math.argmax(true[i],axis=-1,output_type=tf.int32)
        n = n+ tf.reduce_sum(tf.cast((char_pred==char_true)&(tf.convert_to_tensor(i)<num_char_true),tf.int32))
        i=i+1
    accuracy = tf.divide(tf.cast(n,tf.float32) , tf.cast(tf.reduce_sum(num_char_true),tf.float32))
    return accuracy

@tf.function
def custom_loss(y_true,y_pred):
    alpha =5
    y_true = tf.convert_to_tensor(y_true)
    a,b,c,d ,num = tf.split(y_true, [10,10,10,10,4], 1)
    true = [a,b,c,d]
    A,B,C,D,NUM = tf.split(y_pred, [10,10,10,10,4], 1)
    pred = [A,B,C,D]
    num_char_true = tf.math.argmax(num,axis=-1,output_type=tf.int32)+ 1
    loss_1 = keras.losses.categorical_crossentropy(num ,NUM)
    i =0
    loss =tf.zeros_like(loss_1,dtype=tf.float32)
    #loss =tf.zeros([32,1],tf.float32)
    while (i<4) :
        char_pred = pred[i]
        char_true = true[i]
        loss = loss + tf.cast(tf.convert_to_tensor(i)<num_char_true,tf.float32)*keras.losses.categorical_crossentropy(char_true,char_pred)
        i=i+1
    return alpha * tf.reduce_sum(loss_1) + tf.reduce_sum(loss) 

def build_and_compile_model():
    
    rate =0.2
    rate2 = 0.5
    inputs = keras.Input(shape=(128, 128,1), name='img')
    #x = layers.Conv2D(16, 5, activation='relu',kernel_constraint=MaxNorm(3),padding='same')(inputs)
    x = Conv2D(16, 5, activation='relu',padding='same')(inputs)
    x = Dropout(rate)(x)
    x = Conv2D(16, 5, activation='relu',padding='same')(x)
    x = Dropout(rate)(x)
    x = MaxPooling2D(4)(x)
    x = Conv2D(32, 5, activation='relu',padding='same')(x)
    x = Dropout(rate)(x)
    x = Conv2D(32, 5, activation='relu',padding='same')(x)
    x = Dropout(rate)(x)
    x = MaxPooling2D(4)(x)
    x = Conv2D(64, 5, activation='relu',padding='same')(x)
    x = Dropout(rate)(x)
    x = Conv2D(64, 5, activation='relu',padding='same')(x)
    x = Dropout(rate)(x)
    x = MaxPooling2D(4)(x)
   
    feature = Flatten()(x)
    feature = Dropout(rate2)(feature)
    outputs = layers.concatenate([Dense(10, activation='softmax')(feature),Dense(10, activation='softmax')(feature),
                                  Dense(10, activation='softmax')(feature),Dense(10, activation='softmax')(feature),
                                  Dense(4, activation='softmax', name='num_digits')(feature)])
    
    model = keras.Model(inputs=inputs , outputs=outputs, name='mnist_model')
    model.compile(loss=custom_loss,optimizer=keras.optimizers.RMSprop(learning_rate=0.001)
                  ,metrics=[custom_acc,num_char_acc])
    return model


In [ ]:
'''
model = keras.models.load_model('/kaggle/input/recognition-rev/model.h5',compile=False)
model.compile(loss=custom_loss,optimizer=keras.optimizers.RMSprop(learning_rate=0.001)
                  ,metrics=[custom_acc,num_char_acc])
'''
model=build_and_compile_model()
model.load_weights('/kaggle/input/digits-recog/chkpt')


In [ ]:
def parse_func(s_example):
    features = {
                'label':tf.io.FixedLenFeature((44,),tf.float32),
                'image':tf.io.FixedLenFeature((128,128,1),tf.float32),
                }
    example = tf.io.parse_single_example(s_example, features=features)
    return example['image'],example['label']

FILES = '/kaggle/input/variablelength-handwritten-digits/*.tfrecords'
dataset = tf.data.Dataset.list_files(FILES).\
    interleave(tf.data.TFRecordDataset, cycle_length=4, block_length=1).map(parse_func)

train_dataset = dataset.skip(500).shuffle(500).batch(1000).take(1)
a,b =next(iter(train_dataset))
new_data = a.numpy()
new_label = np.stack([np.argmax(b[:,10*i:10*i+10],axis=-1) for i in range(4)],axis=1)
new_count = np.argmax(b[:,40:],axis=-1)

In [ ]:
results = model.predict(new_data)
num_pred = np.argmax(results[:,40:],axis =-1)+1
chars = np.zeros((1000,4),dtype=np.int64)
for i in range(4):
    chars[:,i]= np.argmax(results[:,10*i:10*i+10],axis=-1) 
mask = np.zeros((1000,4),dtype=np.int64)
for j in range(1000):
    mask[j,:new_count[j]+1]=1
results= np.argmax(results,axis=-1)
print("Error ratios:",(mask*(new_label!=chars)).sum() / (new_count.sum()+len(new_data)))

In [ ]:
fig,a =  plt.subplots(3,4,figsize=(16,12))
for i in range(3):
    for j in range(4):
        ID = np.random.randint(len(new_data)),
        a[i][j].axis('off')
        label = [ new_label[ID][x] for x in range(new_count[ID]+1)]
        prediction = [ chars[ID][x] for x in range(num_pred[ID])]
        a[i][j].set_title('Prediction:'+ ''.join([str(x) for x in prediction])+\
                          '\nGround truth:'+''.join([str(x) for x in label]))
        a[i][j].imshow(new_data[ID].squeeze(),cmap='gray')